# Random Forest: Classification by Automation Condition (By Participant Sampling)
- Prediction of automation usage using all features and the top 20
- We train and test by automation condition
    - Grouping the data from all participants in a condition

## Necessary Libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import glob as glob
import os as os
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from joblib import Parallel, delayed
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import *

## Reading and processing the data

### Choose if running on Agave

In [2]:
AGAVE = False


### Choose automation condition

In [293]:
choose_condition  = 3
conditions = ['SH','SL','FH','FL','ALL']
cond = conditions[choose_condition]

### Path to all data files

In [294]:
# Files
if(AGAVE==True):
    files_path = '../../../NewFeatures/' + cond             # Agave
else:
    files_path = '../../../features_data_risk/' + cond      # Local

all_files = glob.glob(os.path.join(files_path, "*.csv"))

### Features that offer extra information

In [295]:
# Features that offer extra information (they're basically the same as boolAuto)
features_extra = ['boolHand','boolTake','brakeOshp','sumAuto','sumHand','sumTake','sumTogg']

### Features that are related to the operator's actions

In [296]:
# Actions
features_internal = ['accAngOshpX','accAngOshpY','accAngOshpZ','accLinOshpX',
					'accLinOshpY','accLinOshpZ','boolButnA','boolButnB',
					'boolViolButn','boolViolLane','boolViolLead','boolViolPeds',
					'boolViolRang','boolViolTraf','boolViolVehs',
					'oriOshpX','oriOshpY','oriOshpZ','psnOshpLane','psnOshpLaneAbs',
					'psnOshpLaneLft','psnOshpX','psnOshpY','psnOshpZ',
					'steerOshp','sumViolButn','sumViolLane','sumViolLead','sumViolPeds',
					'sumViolRang','sumViolTraf','sumViolVehs','throtOshp','timeReact',
					'velAngOshpX','velAngOshpY','velAngOshpZ','velLinOshp','velLinOshpLane',
					'velLinOshpLaneAbs','velLinOshpLaneLft','velLinOshpRang','velLinOshpX',
					'velLinOshpY','velLinOshpZ']

### Read the data files

In [297]:
# Pre-process data
# Assign features from current time-step to future use of automation,
# The idea is to predict future use of automation based on current behavior
delay = 1/60            # How much time in advance we want to predict automation usage (in seconds)
h = (int)(delay*60 - 1) # Turn the time into index (Considering a sampling rate of 60 Hz)
data_list = []
for file in all_files:
    data_participant = pd.read_csv(file)
    data_participant.insert(0,"time",[(i*1/60) for i in range(0,data_participant.shape[0])],True)
    data_auto = data_participant['boolAuto']
    data_auto = np.array(data_auto.iloc[h:])
    data_participant = data_participant.drop('boolAuto', axis=1)
    data_participant = data_participant.head(data_participant.shape[0]-h)
    data_participant.insert(data_participant.shape[1],"boolAuto",data_auto,True)
    data_list.append(data_participant)

### Results of feature selection

In [298]:
SH_feats = ['psnOshpRang'    ,
'score'          ,
'velLinOshpX'    ,
'velLinOshp'     ,
'rrisk'          ,
'sumViolLane'    ,
'velLinLead'     ,
'throtOshp'      ,
'sumViolRang'    ,
'odomRoad'       ,
'velLinOshpRang' ,
'sumViolPeds'    ,
'ttcLead'        ,
'sumViolButn'    ,
'accLinOshpX'    ,
'psnTrafPrxY'    ,
'psnTrafPrxX'    ,
'boolStatRang'   ,
'oriTrafPrxZ'    ,
'oriOshpY'       ]

SL_feats = ['psnOshpRang'    ,
'rrisk'          ,
'sumViolAwrd'    ,
'velLinOshp'     ,
'throtOshp'      ,
'velLinOshpX'    ,
'sumViolRang'    ,
'score'          ,
'accLinOshpX'    ,
'sumViolButn'    ,
'oriOshpY'       ,
'velLinLead'     ,
'odomRoad'       ,
'sumViolLane'    ,
'velLinOshpRang' ,
'psnTrafPrxX'    ,
'psnTrafPrxY'    ,
'accAngOshpY'    ,
'oriLeadZ'       ,
'velAngOshpY'    ]

FH_feats = ['velLinOshpX'    ,
'psnOshpRang'    ,
'score'          ,
'rrisk'          ,
'velLinOshp'     ,
'throtOshp'      ,
'sumViolRang'    ,
'velLinLead'     ,
'odomRoad'       ,
'sumViolButn'    ,
'sumViolLane'    ,
'accLinOshpX'    ,
'velLinOshpRang' ,
'psnOshpLaneLft' ,
'oriTrafPrxZ'    ,
'velAngLeadZAbs' ,
'psnTrafPrxX'    ,
'oriOshpY'       ,
'psnTrafPrxY'    ,
'oriOshpZ'       ]

FL_feats = ['psnOshpRang' ,
'sumViolAwrd' ,
'velLinOshpX' ,
'rrisk'       ,
'throtOshp'   ,
'velLinOshp'  ,
'sumViolLane' ,
'score'       ,
'accLinOshpX' ,
'odomRoad'    ,
'sumViolRang' ,
'oriTrafPrxZ' ,
'sumViolButn' ,
'psnTrafPrxX' ,
'velLinLead'  ,
'sumViolPeds' ,
'psnLeadY'    ,
'timeReact'   ,
'psnTrafPrxY' ,
'psnRoadY'    ]

top_features_all = [SH_feats,SL_feats,FH_feats,FL_feats]
top_feat_cond = top_features_all[choose_condition]


## Evaluate the classification models

In [299]:
# Save the performance results
data_scores = {'Train_Acc':[],'Train_BalAcc':[],'Train_Prec':[],'Train_Rec':[],'Train_Spec':[],
                'Train_AUC':[],'Test_Acc':[],'Test_BalAcc':[],'Test_Prec':[],'Test_Rec':[],
                'Test_Spec':[],'Test_AUC':[]}
perf_scores = pd.DataFrame(data=data_scores)
# Save the ROC curve data
data_roc = pd.DataFrame({'mean_fpr':[], 'tpr_1':[], 'tpr_2':[], 'tpr_3':[], 'tpr_4':[], 
                        'tpr_5':[]}) # 5 KFold
# Save the best set of parameter for the model
best_parameters = pd.DataFrame({'n_estimators':[], 'max_depth':[], 'max_features':[]})

### Random Forest

#### Cross Validation

In [300]:
# Get the best parameters fro random grid search
SEED = 10
n_estimators = [120, 890, 340, 890]
max_depth = [242, 135, 457, 100]
max_features = [15, 10, 10, 10]
best_n_estim = n_estimators[choose_condition]
best_max_depth = max_depth[choose_condition]
best_max_features = max_features[choose_condition]

# Indices of participants
idx_part = list(range(0, 16))
idx_label = [0]*8 + [1]*8

# Create the k-fold
cv = StratifiedKFold(n_splits=5)
# Create the classifier
# classifier = RandomForestClassifier(n_estimators=best_n_estim, max_depth=best_max_depth,
#                             max_features=best_max_features, n_jobs=-1)
# classifier = KNeighborsClassifier(n_jobs=-1)
classifier = make_pipeline(StandardScaler(),LinearSVC())

# Save performance metrics
tprs = []; aucs = []; acc = []; acc_bal = []; prec = []; rec = []; spec = []; AUC_v = []
acc_tr = []; acc_bal_tr = []; prec_tr = []; rec_tr = []; spec_tr = []; AUC_v_tr = []
mean_fpr = np.linspace(0, 1, 100)

# Apply cross-validation
for _, (train, test) in enumerate(cv.split(idx_part,idx_label)):
    train_list = [data_list[index] for index in train]
    test_list = [data_list[index] for index in test]
    # Read the data from the participants to be used for training and testing
    df_data_train = pd.concat(train_list, ignore_index=True)
    df_data_test = pd.concat(test_list, ignore_index=True)
    # Delete the features that offer extra information from the dataset
    df_data_train = df_data_train.drop(features_extra,axis = 1)
    df_data_train = df_data_train.drop('time',axis=1)          # Drop the inserted time too
    df_data_test = df_data_test.drop(features_extra,axis = 1)
    df_data_test = df_data_test.drop('time',axis=1)
    
    # Check if running on Agave
    if(AGAVE==True):
        sample_train = df_data_train.sample(frac=1, random_state=SEED);
        sample_test = df_data_test.sample(frac=1, random_state=SEED);
    else:
        sample_train = df_data_train.sample(frac=0.1, random_state=SEED);
        sample_test = df_data_test.sample(frac=0.1, random_state=SEED);
    
    # Scale the data to a range [0,1]
    idx_train = np.shape(sample_train)[0]
    sample_comb = MinMaxScaler(copy=False).fit_transform(np.concatenate((sample_train[top_feat_cond].to_numpy(), 
                                    sample_test[top_feat_cond].to_numpy()), axis=0))
    # Split scaled data into training and testing
    X_train = sample_comb[0:idx_train]
    y_train = sample_train['boolAuto']
    X_test = sample_comb[idx_train:len(sample_comb)]
    y_test = sample_test['boolAuto']

    # Fit the model with the given training set
    classifier.fit(X_train, y_train)
    
    # Get predictions for the training and testing sets
    y_pred = classifier.predict(X_test)
    y_pred_tr = classifier.predict(X_train)

    # ROC curve metrics
    fpr, tpr, _ = roc_curve(y_test, y_pred, pos_label=1)
    interp_tpr = np.interp(mean_fpr, fpr, tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)

    # Other performance metrics
    # Testing
    acc.append(accuracy_score(y_test,y_pred))
    acc_bal.append(balanced_accuracy_score(y_test,y_pred))
    prec.append(precision_score(y_test,y_pred))
    rec.append(recall_score(y_test,y_pred, pos_label=1))
    spec.append(recall_score(y_test,y_pred, pos_label=0))
    AUC_v.append(roc_auc_score(y_test,y_pred))
    # Training
    acc_tr.append(accuracy_score(y_train,y_pred_tr))
    acc_bal_tr.append(balanced_accuracy_score(y_train,y_pred_tr))
    prec_tr.append(precision_score(y_train,y_pred_tr))
    rec_tr.append(recall_score(y_train,y_pred_tr, pos_label=1))
    spec_tr.append(recall_score(y_train,y_pred_tr, pos_label=0))
    AUC_v_tr.append(roc_auc_score(y_train,y_pred_tr))


c:\Users\cbust\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
c:\Users\cbust\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
c:\Users\cbust\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
c:\Users\cbust\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
c:\Users\cbust\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iter

#### Save the results

In [301]:
# Fill the dataframe with all the results
perf_scores.Train_Acc = acc_tr
perf_scores.Train_BalAcc = acc_bal_tr
perf_scores.Train_Prec = prec_tr
perf_scores.Train_Rec = rec_tr
perf_scores.Train_Spec = spec_tr
perf_scores.Train_AUC = AUC_v_tr
perf_scores.Test_Acc = acc
perf_scores.Test_BalAcc	 = acc_bal
perf_scores.Test_Prec = prec
perf_scores.Test_Rec = rec
perf_scores.Test_Spec = spec
perf_scores.Test_AUC = AUC_v
# Best parameters
best_parameters.n_estimators = [best_n_estim]
best_parameters.max_depth = [best_max_depth]
best_parameters.max_features = [best_max_features]
# Data ROC curves
data_roc.mean_fpr = mean_fpr
for i in range(1,6):
    data_roc.iloc[:,i] = tprs[i-1]

In [302]:
perf_scores

,Train_Acc,Train_BalAcc,Train_Prec,Train_Rec,Train_Spec,Train_AUC,Test_Acc,Test_BalAcc,Test_Prec,Test_Rec,Test_Spec,Test_AUC
0,0.774490,0.706087,0.717232,0.509368,0.902806,0.706087,0.690195,0.575670,0.467553,0.292593,0.858747,0.575670
1,0.739994,0.599915,0.623735,0.265424,0.934407,0.599915,0.764554,0.755793,0.750286,0.688112,0.823474,0.755793
2,0.747231,0.668167,0.671016,0.440912,0.895422,0.668167,0.794759,0.670632,0.742515,0.393310,0.947953,0.670632
3,0.767956,0.677160,0.730637,0.427837,0.926483,0.677160,0.686276,0.548539,0.528727,0.166953,0.930125,0.548539
4,0.757572,0.670217,0.731356,0.415123,0.925310,0.670217,0.724549,0.611001,0.451533,0.379556,0.842446,0.611001


### 91 Features

In [303]:
# Get the best parameters from random grid search
SEED = 10
n_estimators = [120, 890, 340, 890]
max_depth = [242, 135, 457, 100]
max_features = [15, 10, 10, 10]
best_n_estim = n_estimators[choose_condition]
best_max_depth = max_depth[choose_condition]
best_max_features = max_features[choose_condition]

# Indices of participants
idx_part = list(range(0, 16))
idx_label = [0]*8 + [1]*8

# Create the k-fold
cv = StratifiedKFold(n_splits=5)
# Create the classifier
# classifier = RandomForestClassifier(n_estimators=best_n_estim, max_depth=best_max_depth,
#                             max_features=best_max_features, n_jobs=-1)
# classifier = KNeighborsClassifier(n_jobs=-1)
classifier = make_pipeline(StandardScaler(),LinearSVC())

# Save performance metrics
tprs = []; aucs = []; acc = []; acc_bal = []; prec = []; rec = []; spec = []; AUC_v = []
acc_tr = []; acc_bal_tr = []; prec_tr = []; rec_tr = []; spec_tr = []; AUC_v_tr = []
mean_fpr = np.linspace(0, 1, 100)

# Apply cross-validation
for _, (train, test) in enumerate(cv.split(idx_part,idx_label)):
    train_list = [data_list[index] for index in train]
    test_list = [data_list[index] for index in test]
    # Read the data from the participants to be used for training and testing
    df_data_train = pd.concat(train_list, ignore_index=True)
    df_data_test = pd.concat(test_list, ignore_index=True)
    # Delete the features that offer extra information from the dataset
    df_data_train = df_data_train.drop(features_extra,axis = 1)
    df_data_train = df_data_train.drop('time',axis=1)          # Drop the inserted time too
    df_data_test = df_data_test.drop(features_extra,axis = 1)
    df_data_test = df_data_test.drop('time',axis=1)
    
    # Check if running on Agave
    if(AGAVE==True):
        sample_train = df_data_train.sample(frac=1, random_state=SEED);
        sample_test = df_data_test.sample(frac=1, random_state=SEED);
    else:
        sample_train = df_data_train.sample(frac=0.1, random_state=SEED);
        sample_test = df_data_test.sample(frac=0.1, random_state=SEED);
    
    # Scale the data to a range [0,1]
    idx_train = np.shape(sample_train)[0]
    sample_comb = MinMaxScaler(copy=False).fit_transform(np.concatenate((
        sample_train.drop('boolAuto',axis=1).to_numpy(), 
        sample_test.drop('boolAuto',axis=1).to_numpy()), axis=0))
    # Split scaled data into training and testing
    X_train = sample_comb[0:idx_train]
    y_train = sample_train['boolAuto']
    X_test = sample_comb[idx_train:len(sample_comb)]
    y_test = sample_test['boolAuto']

    # Fit the model with the given training set
    classifier.fit(X_train, y_train)
    
    # Get predictions for the training and testing sets
    y_pred = classifier.predict(X_test)
    y_pred_tr = classifier.predict(X_train)

    # ROC curve metrics
    fpr, tpr, _ = roc_curve(y_test, y_pred, pos_label=1)
    interp_tpr = np.interp(mean_fpr, fpr, tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)

    # Other performance metrics
    # Testing
    acc.append(accuracy_score(y_test,y_pred))
    acc_bal.append(balanced_accuracy_score(y_test,y_pred))
    prec.append(precision_score(y_test,y_pred))
    rec.append(recall_score(y_test,y_pred, pos_label=1))
    spec.append(recall_score(y_test,y_pred, pos_label=0))
    AUC_v.append(roc_auc_score(y_test,y_pred))
    # Training
    acc_tr.append(accuracy_score(y_train,y_pred_tr))
    acc_bal_tr.append(balanced_accuracy_score(y_train,y_pred_tr))
    prec_tr.append(precision_score(y_train,y_pred_tr))
    rec_tr.append(recall_score(y_train,y_pred_tr, pos_label=1))
    spec_tr.append(recall_score(y_train,y_pred_tr, pos_label=0))
    AUC_v_tr.append(roc_auc_score(y_train,y_pred_tr))


c:\Users\cbust\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
c:\Users\cbust\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
c:\Users\cbust\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
c:\Users\cbust\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
c:\Users\cbust\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\svm\_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iter

In [304]:
np.sum(y_test==1)/len(y_test)

0.25469775865972377

In [305]:
# Fill the dataframe with all the results
perf_scores.Train_Acc = acc_tr
perf_scores.Train_BalAcc = acc_bal_tr
perf_scores.Train_Prec = prec_tr
perf_scores.Train_Rec = rec_tr
perf_scores.Train_Spec = spec_tr
perf_scores.Train_AUC = AUC_v_tr
perf_scores.Test_Acc = acc
perf_scores.Test_BalAcc	 = acc_bal
perf_scores.Test_Prec = prec
perf_scores.Test_Rec = rec
perf_scores.Test_Spec = spec
perf_scores.Test_AUC = AUC_v
# Best parameters
best_parameters.n_estimators = [best_n_estim]
best_parameters.max_depth = [best_max_depth]
best_parameters.max_features = [best_max_features]
# Data ROC curves
data_roc.mean_fpr = mean_fpr
for i in range(1,6):
    data_roc.iloc[:,i] = tprs[i-1]

In [306]:
perf_scores

,Train_Acc,Train_BalAcc,Train_Prec,Train_Rec,Train_Spec,Train_AUC,Test_Acc,Test_BalAcc,Test_Prec,Test_Rec,Test_Spec,Test_AUC
0,0.824168,0.767775,0.807112,0.605597,0.929953,0.767775,0.715585,0.600933,0.537832,0.317540,0.884325,0.600933
1,0.787151,0.680687,0.728562,0.426462,0.934911,0.680687,0.792253,0.768697,0.901666,0.586713,0.950681,0.768697
2,0.794962,0.736629,0.742004,0.568964,0.904294,0.736629,0.684533,0.551781,0.391074,0.255190,0.848371,0.551781
3,0.800842,0.730729,0.765734,0.538204,0.923255,0.730729,0.742519,0.646369,0.671577,0.379995,0.912743,0.646369
4,0.802314,0.736912,0.787629,0.545925,0.927899,0.736912,0.723719,0.658034,0.462605,0.524148,0.791920,0.658034
